In [43]:
# Calculating vtract and vcore
#
# Data input
# Potential sites
# NLCD
# AOI

# Methods
# Clip woody with AOI
# Iterate over potential sites and select woody within 10m.  Merge these
# Calculate area.  This is vtract
# Internal buffer by 100.  This is vcore.
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
from rasterio.mask import mask
from shapely.geometry import box, shape

In [44]:
savefile = 'exampleprojects'

workspace = r'D:\gis\projects\YazooHGM\exampleprojects'
aoi = os.path.join(workspace,'yazoo_study_area.shp')
#woody = os.path.join(workspace, 'WoodyWetlandsPoly.shp')
foi = os.path.join(workspace, 'examples.shp')

In [45]:
aoi = gpd.read_file(aoi).to_crs(5070)
#woody = gpd.read_file(woody).to_crs(5070)
foi = gpd.read_file(foi).to_crs(5070)

In [53]:
cogin = 'https://giscog.blob.core.windows.net/newcontainer/nlcd2019_cog.tif'
cog = rasterio.open(cogin)
print(cog.crs)
aoicrs = aoi.to_crs(cog.crs)
nlcd, nlcd_transform =rasterio.mask.mask(cog, aoicrs.dissolve().geometry, crop=True)
ag = nlcd.copy()
ag[ag>82]=0
ag[ag<81]=0
ag[ag==81]=1
ag[ag==82]=1
shapes = rasterio.features.shapes(ag[0], transform=nlcd_transform, mask=ag[0] == 1)
ag = gpd.GeoDataFrame({'geometry': [shape(geom) for geom, value in shapes]})
ag = ag.set_geometry('geometry')
ag = ag.set_crs(5070)
woody = nlcd.copy()
woody[(woody>90) & (woody <90)]=0
woody[woody==90]=1
shapes = rasterio.features.shapes(woody[0], transform=nlcd_transform, mask=woody[0] == 1)
woody = gpd.GeoDataFrame({'geometry': [shape(geom) for geom, value in shapes]})
woody = woody.set_geometry('geometry')
woody = woody.set_crs(5070)

PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [54]:
clippedwoody = gpd.clip(woody, aoi)

In [55]:
foi = gpd.clip(foi, ag)
foi['uid'] = foi.index.map(hash)
foi['acres'] = foi.area*0.000247105

In [57]:
'''
import folium
m = foi.explore()
#woodypoly.explore(m=m, color='yellow')
clippedwoody.explore(m=m, color='purple')
folium.LayerControl().add_to(m) 
m
'''

"\nimport folium\nm = foi.explore()\n#woodypoly.explore(m=m, color='yellow')\nclippedwoody.explore(m=m, color='purple')\nfolium.LayerControl().add_to(m) \nm\n"

In [62]:
hold = gpd.GeoDataFrame()
for ft in foi.iterfeatures():
    ws = gpd.GeoDataFrame.from_features([ft],crs="EPSG:5070")
    wsbuffer = ws.copy()
    wsbuffer['geometry'] = ws['geometry'].buffer(10)
    selectwoody = gpd.sjoin(clippedwoody[['geometry']],wsbuffer, how='inner',predicate='intersects')
    ws = gpd.overlay(ws, selectwoody, how='union')
    ws = ws.dissolve()
    hold = pd.concat([hold, ws])
hold['vtract'] = hold.area*0.000247105
hold['vcore'] = hold['geometry'].buffer(-100).area*0.000247105
hold = hold.drop(columns=['index_right'])

D:\mambaforge\envs\newduckdb\lib\site-packages\geopandas\geodataframe.py:1815: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union
C:\Users\mmitchell.DUCKS\AppData\Local\Temp\ipykernel_6368\507345203.py:7: UserWarning: `keep_geom_type=True` in overlay resulted in 14 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  ws = gpd.overlay(ws, selectwoody, how='union')
D:\mambaforge\envs\newduckdb\lib\site-packages\geopandas\geodataframe.py:1815: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union
C:\Users\mmitchell.DUCKS\AppData\Local\Temp\ipykernel_6368\507345203.py:7: UserWarning: `keep_geom_type=True` in overlay resulted in 18 dropped geometries of different geometry types than df

In [70]:
'''
import folium
m = foi.explore()
#woodypoly.explore(m=m, color='yellow')
hold.explore(m=m, color='purple')
folium.LayerControl().add_to(m) 
m
'''
hold = hold.rename(columns={'Name_1': 'Name', 'uid_1': 'uid', 'acres_1': 'acres'}).drop(columns=['acres_2', 'uid_2'])
hold

,geometry,Name,uid,acres,Name_2,vtract,vcore
0,"POLYGON Z ((470115.000 1057305.000 0.000, 4701...",HGM example 5,4.0,1483.437432,HGM example 5,7869.675960,5425.213093
0,MULTIPOLYGON Z (((483585.000 1061565.000 0.000...,HGM example 3,2.0,444.340256,HGM example 3,1442.891561,726.973783
0,"POLYGON Z ((466995.000 1061865.000 0.000, 4669...",HGM example 4,3.0,719.881539,HGM example 4,6631.349743,5283.477363
0,MULTIPOLYGON Z (((481155.000 1069725.000 0.000...,HGM example 2,1.0,1257.469609,HGM example 2,11856.124296,8818.632729
0,MULTIPOLYGON Z (((483698.967 1078821.914 0.000...,HGM example 1,0.0,984.346236,HGM example 1,9416.211309,6963.742716


In [71]:
#import folium
bigbuffer = hold.copy()
bigbuffer['geometry'] = bigbuffer.buffer(500).boundary
bigbuffer['perim'] = bigbuffer.length
#m = bigbuffer.explore(name='buffer', color='blue')
#foi.explore(m=m, name='tract', color='black')
bigbuffer = bigbuffer.overlay(clippedwoody, how='intersection')
bigbuffer = bigbuffer.dissolve(by='uid', aggfunc={'perim':'first'})
#bigbuffer.explore(m=m, name='overlay', color='red')
#clippedwoody.explore(m=m, name='woody', color='green')
bigbuffer['connectperim'] = bigbuffer.length
bigbuffer['vconnect'] = (bigbuffer['connectperim']/bigbuffer['perim'])*100
#folium.LayerControl().add_to(m)
#m

In [72]:
bigbuffer = bigbuffer.reset_index()
hold = hold.merge(bigbuffer[['uid', 'vconnect']], on='uid')
#hold['vconnect'] = hold['vconnect_y']
#hold = hold.drop(['vconnect_x','vconnect_y', 'vcore_acr'], axis=1)

,uid,geometry,perim,connectperim,vconnect
0,0.0,"MULTILINESTRING ((488775.000 1075968.268, 4887...",38860.487828,26663.531692,68.613477
1,1.0,"MULTILINESTRING ((485295.000 1066353.488, 4852...",35769.468081,9831.444258,27.485576
2,2.0,"MULTILINESTRING ((482985.000 1060830.099, 4829...",17321.132271,2342.720069,13.525213
3,3.0,"MULTILINESTRING ((468434.331 1070025.000, 4684...",31062.888724,13867.276061,44.642584
4,4.0,"MULTILINESTRING ((472005.000 1054293.488, 4719...",33715.721930,16204.337408,48.061665


,geometry,Name,uid,acres,Name_2,vtract,vcore,vconnect
0,"POLYGON Z ((470115.000 1057305.000 0.000, 4701...",HGM example 5,4.0,1483.437432,HGM example 5,7869.675960,5425.213093,48.061665
1,MULTIPOLYGON Z (((483585.000 1061565.000 0.000...,HGM example 3,2.0,444.340256,HGM example 3,1442.891561,726.973783,13.525213
2,"POLYGON Z ((466995.000 1061865.000 0.000, 4669...",HGM example 4,3.0,719.881539,HGM example 4,6631.349743,5283.477363,44.642584
3,MULTIPOLYGON Z (((481155.000 1069725.000 0.000...,HGM example 2,1.0,1257.469609,HGM example 2,11856.124296,8818.632729,27.485576
4,MULTIPOLYGON Z (((483698.967 1078821.914 0.000...,HGM example 1,0.0,984.346236,HGM example 1,9416.211309,6963.742716,68.613477


In [75]:
outfoi = foi.merge(hold[['uid', 'vtract', 'vcore', 'vconnect']], on='uid')

,Name,geometry,uid,acres,vtract,vcore,vconnect
0,HGM example 5,MULTIPOLYGON Z (((470325.000 1060515.000 0.000...,4,1483.437432,7869.675960,5425.213093,48.061665
1,HGM example 3,MULTIPOLYGON Z (((483465.000 1063575.000 0.000...,2,444.340256,1442.891561,726.973783,13.525213
2,HGM example 4,MULTIPOLYGON Z (((467745.000 1064475.000 0.000...,3,719.881539,6631.349743,5283.477363,44.642584
3,HGM example 2,MULTIPOLYGON Z (((481755.000 1071515.387 0.000...,1,1257.469609,11856.124296,8818.632729,27.485576
4,HGM example 1,MULTIPOLYGON Z (((480495.000 1078665.000 0.000...,0,984.346236,9416.211309,6963.742716,68.613477


In [76]:
outfoi.to_parquet(os.path.join(workspace,savefile + '_sitesv2.parquet'))